In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=21)
svr = SVR(kernel = 'rbf')
parameters = {'C':[0.2,0.5,1,2],'gamma':[0.01,0.05,0.1]}
model = GridSearchCV(svr,param_grid = parameters, cv=5)
model.fit(X_train,y_train)
print('The best score is %.3f'%model.best_score_)
print('The best parameters are: ',model.best_params_)
print('The test set score after tuning parameters is %.3f'%model.score(X_test,y_test))

In [ ]:
# In Story Telling section, it was shown that the correlations between poverty and majority of rural urban continuum codes 
# i.e. all except RUCC 1 and 6, are close to each other . Therefore, any of these seven RUCC (2-5,7-9) could be picked as the 
# base to simplify the interpretation. I pick RUCC 9 and remove it

In [ ]:
# Build the linear model with OLS command
X2 = sm.add_constant(X) #OLS by default does not have intercept
model = sm.OLS(y,X2).fit()

# Remove predcitors with p-value less than 0.001
X_new = X2.loc[:,model.pvalues[model.pvalues <0.001].index]

# fit the model on the dataset with reduced number of predictors
model = sm.OLS(y,X_new).fit()

# print summary of the model
model.summary()

# Remove predcitors with p-value less than 0.001
X_new = X2.loc[:,model.pvalues[model.pvalues <0.001].index]

# fit the model on the dataset with reduced number of predictors
model = sm.OLS(y,X_new).fit()

# print summary of the model
model.summary()

# Plot residuals vs. fitted values
figure,ax = plt.subplots(figsize=(10,6))
ax.scatter(model.fittedvalues,model.resid)
ax.set_xlabel('Fitted values')
ax.set_ylabel('Residuals');
ax.set_title('Residuals vs. fitted values',y=1.02)
ax.axis([0,45,-35,35])

# Plot quantile plot
figure,ax = plt.subplots(figsize=(10,6))
stats.probplot(model.resid,plot=ax)
ax.set_ylabel('Residual quantiles')
ax.set_title('Quantile-Quantile plot',y=1.02)
ax.axis([-4,4,-35,35])

# Show plots
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=21)
svr = SVR(kernel='linear')
parameters = {'C':[2,5,10],\
             'epsilon':[0.5,0.8]}
model = GridSearchCV(svr,param_grid = parameters, cv=5)
model.fit(X_train_extended,y_train)
print('The test set score after tuning parameters is ',np.round(model.score(X_test_extended,y_test),3))
print('The best parameters are: ',model.best_params_)

In [ ]:
# Define the influence array and create a panda peries with the external studentized residuals
infl = model.get_influence()
p = pd.Series(infl.resid_studentized_external)

# Extract the indices of the series for which the absolute value of external studentized residual is more than 3.
indices = p[np.abs(p)>3].index

# Create a dataframe from the original dataframe which only include the indices extracted above
df_outliers = df.iloc[indices]
X_reduced=X_new.iloc[p[np.abs(p)<=3].index]
y_reduced=y.iloc[p[np.abs(p)<=3].index]

In [1]:
# Import DummyRegressor
from sklearn.dummy import DummyRegressor

# Build the dummy classifier
clf = DummyRegressor(strategy = 'mean')

In [ ]:
# Import necessary libraries
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Create linear regression classifier and polynomial features, and use pipeline
linear = LinearRegression()
poly = PolynomialFeatures(degree = 2, interaction_only = True)
pipeline = make_pipeline(poly, linear)

# Run cross-validation with R-squared as scoring and print the results
scores = cross_val_score(pipeline,X_reduced,y_reduced,cv=5)
print('\nThe average of 5-fold cross validation score (R-squared) for the simplified linear model is ', np.round(np.mean(scores),2))

# Run cross-validation with mean-squared-error as scoring and print the results
scores = cross_val_score(pipeline,X_reduced,y_reduced,cv=5,scoring='neg_mean_squared_error')
print('The average of 5-fold cross validation score (mean-squared-error) for the simplified linear model is ', np.round(-np.mean(scores),2))

In [ ]:
# Create the dataframe for dummy variables representing the nine RUCC and concat it to the original dataframe
ohe = OneHotEncoder()
RUCC_matrix = ohe.fit_transform(df.RUCC.values.reshape(-1,1)).toarray()
RUCC_df = pd.DataFrame(RUCC_matrix,columns = ['RUCC_'+str(i) for i in np.arange(1,10)])
df_concat = pd.concat([df_copy,RUCC_df],axis = 1)

In [ ]:
#Order the data frame based on South, Northeast, Midwest, and West.
df.region = pd.Categorical(df.region, categories=["South","Northeast","Midwest","West"],ordered=True)
df.sort_values('region',inplace=True)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_concat[df_concat['2013 Rural-urban Continuum Code'].isnull()].reset_index(drop=True))

In [9]:
df=pd.read_csv('wrangled_data.csv')
df.RUCC = df.RUCC.astype('int').astype('str')

def new_RUCC(row):
    if row['RUCC'] in ['4','5']:
        k = '4_or_5'
    elif row['RUCC'] in ['6','7']:
        k = '6_or_7'
    elif row['RUCC'] in ['8','9']:
        k = '8_or_9'
    else:
        k = row['RUCC']
    return k

#print(df.head())
df['new_RUCC']=df.apply(new_RUCC,axis=1)
df.head()

   height  width
0    40.0     10
1    20.0      9
2     3.4      4
   height  width   area
0    40.0     10  400.0
1    20.0      9  180.0
2     3.4      4   13.6


In [5]:
import seaborn as sns
flights = sns.load_dataset("flights")
flights.head()

C:\Users\moein\Miniconda3\envs\mypy36\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [6]:
flights = flights.pivot("month", "year", "passengers")
flights.head()

year,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960
month,,,,,,,,,,,,
January,112,115,145,171,196,204,242,284,315,340,360,417
February,118,126,150,180,196,188,233,277,301,318,342,391
March,132,141,178,193,236,235,267,317,356,362,406,419
April,129,135,163,181,235,227,269,313,348,348,396,461
May,121,125,172,183,229,234,270,318,355,363,420,472


In [ ]:
# Group bu RUCC and measure the average
df_group = df.groupby('RUCC').mean().loc[:,['high_school_or_less','poverty','unemployment']]

# Plot the average of less than high school, poverty, and unemployment across the nine RUCC
plt.plot(df_group,'--D')
plt.legend(df_group.columns,loc='lower left',bbox_to_anchor=(1,0))
plt.xlabel('RUCC',fontsize = 20)
plt.ylabel('percentage',fontsize = 20)
plt.title('Average percentage of population with high school diploma or less,\n \
poverty and unemployment across nine RUCC',fontsize = 20)
plt.show()